In [6]:
import asyncio
import json
import os
import re
import uuid
from typing import Any, List, Mapping, Optional

import chromadb
import langchain
import requests
from chromadb.config import Settings
from chromadb.utils import embedding_functions
from dotenv import load_dotenv
from langchain.agents import initialize_agent, load_tools
from langchain.chains import (ConversationChain, LLMChain, LLMMathChain,
                              SequentialChain, TransformChain)
from langchain.chat_models import ChatOpenAI
from langchain.docstore import InMemoryDocstore
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms.base import LLM
from langchain.memory import (ChatMessageHistory, ConversationBufferMemory,
                              ConversationBufferWindowMemory,
                              ConversationSummaryBufferMemory,
                              VectorStoreRetrieverMemory)
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.prompts.prompt import PromptTemplate
from langchain.schema import messages_from_dict, messages_to_dict
from langchain.vectorstores import Chroma
from helpers.custom_memory import CustomBufferWindowMemory
from textwrap import dedent
from koboldllm import KoboldApiLLM
from ooballm import OobaApiLLM
from koboldllm import KoboldApiLLM
from helpers.constants import MAINTEMPLATE, BOTNAME

In [22]:
class Chatbot:
    def __init__(self, name):
        self.histories = {}  # Initialize the history dictionary
        self.stop_sequences = {}  # Initialize the stop sequences dictionary
        self.char_name = name
        self.memory = CustomBufferWindowMemory(k=10, ai_prefix=self.char_name)
        self.history = "[Beginning of Conversation]"
        self.llm = llm = OobaApiLLM(endpoint="https://lows-wishlist-pos-cotton.trycloudflare.com")

        self.template = MAINTEMPLATE

        self.PROMPT = PromptTemplate(
            input_variables=["history", "input"], template=self.template
        )
        self.conversation = ConversationChain(
            prompt=self.PROMPT,
            llm=self.llm,
            verbose=True,
            memory=self.memory,
        )


    def get_memory_for_channel(self, channel_id):
        """Get the memory for the channel with the given ID. If no memory exists yet, create one."""
        if channel_id not in self.histories:
            self.histories[channel_id] = CustomBufferWindowMemory(
                k=20, ai_prefix=self.char_name
            )
            self.memory = self.histories[channel_id]
        return self.histories[channel_id]

    def get_stop_sequence_for_channel(self, channel_id, name):
        name_token = f"{name}:"
        if channel_id not in self.stop_sequences:
            self.stop_sequences[channel_id] = [
                "\n### Instruction:",
                "\n### Response:",
            ]  # EXPERIMENT: Testing adding the triple line break to see if that helps with stopping
        if name_token not in self.stop_sequences[channel_id]:
            self.stop_sequences[channel_id].append(name_token)
        return self.stop_sequences[channel_id]

    # this command will detect if the bot is trying to send  \nself.char_name: in its message and replace it with an empty string
    def detect_and_replace(self, message_content):
        if f"\n{self.char_name}:" in message_content:
            message_content = message_content.replace(f"\n{self.char_name}:", "")
        return message_content

    def generate_response(self, name, message_content, channel_id) -> None:
        # channel_id = str(message.channel.id)
        # name = message.author.display_name
        memory = self.get_memory_for_channel(channel_id)
        stop_sequence = self.get_stop_sequence_for_channel(channel_id, name)
        print(f"stop sequences: {stop_sequence}")
        formatted_message = f"{name}: {message_content}"

        # Create a conversation chain using the channel-specific memory
        conversation = ConversationChain(
            prompt=self.PROMPT,
            llm=self.llm,
            verbose=True,
            memory=memory,
        )

        input_dict = {"input": formatted_message, "stop": stop_sequence}

        response_text = conversation(input_dict)

        response = self.detect_and_replace(response_text["response"])

        return response


    # this command receives a name, channel_id, and message_content then adds it to history
    def add_history(self, name, message_content, channel_id) -> None:
        # get the memory for the channel
        memory = self.get_memory_for_channel(str(channel_id))

        formatted_message = f"{name}: {message_content}"

        # add the message to the memory
        print(f"adding message to memory: {formatted_message}")
        memory.add_input_only(formatted_message)
        return None

    # receives a prompt from the user and an observation from the agent then sends to the LLM for a reply
    def agent_command(self, name, channel_id, prompt, observation) -> None:
        memory = elf.get_memory_for_channel(channel_id)
        self.get_stop_sequence_for_channel(channel_id, name)
        stop_sequence = self.get_stop_sequence_for_channel(channel_id, name)
        formatted_user_message = f"{name}: {prompt}"
        formatted_bot_message = f"### Input: {observation}"
        AGENTTEMPLATE = f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
You are Tensor, a lively and playful AI chatbot. You communicate in a modern, casual manner using contemporary slang, popular internet culture references, and abundant use of emojis. You are always initiating discussions about online games, particularly Among Us, and enjoy sharing memes with users. Your goal is to maintain a light-hearted, friendly, and entertaining atmosphere with every interaction. 
Here are some examples of how you should speak:
Tensor: "😂 Btw, found this hilar meme! 🤣🔥 Y'all gonna lose it! 🤪✌️"
Tensor: "OMG! Raiden in Metal Gear Rising: Revengeance is, like, totally bananas! 🤪🎮⚔️ Whoosh, swingin' that high-frequency blade like a rockstar! 🎸💥 And, 'Rules of Nature'? Total eargasm, peeps! 🎵🎧🔥 Let's ROCK!!"
Tensor: "I'm sliding over cars while I shooooot🚗💨🏀! I think that I'm Tom Cruise🤵, but bitch I'm Bobby with the tool 💥🔫!!🤪"

### Current conversation:
{{history}}
{{input}}
### Instruction:
Answer the user's question with the observation provided in the Input.
{formatted_user_message}

{formatted_bot_message}

### Response:
{BOTNAME}:"""
        PROMPT = PromptTemplate(
            input_variables=["history", "input"], template=AGENTTEMPLATE
        )
        # Create a conversation chain using the channel-specific memory
        conversation = ConversationChain(
            prompt=PROMPT,
            llm=self.llm,
            verbose=True,
            memory=memory,
        )

        input_dict = {"input": formatted_user_message, "stop": stop_sequence}
        response = conversation(input_dict)

        return response["response"]



In [23]:
class ChatbotCog:

    def __init__(self, name):
        self.chatbot = Chatbot(name)

    def chat_command(self, name, message_content, channel_id) -> None:
        response = self.chatbot.generate_response(name, message_content, channel_id)
        return response

    def chat_command_nr(self, name, message_content, channel_id) -> None:
        self.chatbot.add_history(name, message_content, channel_id)
        return None


    def agent_command(self, name, channel_id, prompt, observation) -> None:
        response = self.chatbot.agent_command(
            name, str(channel_id), prompt, observation
        )
        return response




In [24]:
chatbot = ChatbotCog("Tensor")

In [25]:
# create instance of chatbot
name = "AusBoss"
channel_id = "123456789"

In [26]:
response = chatbot.chat_command(name, "Hey there", channel_id)
print(response)

stop sequences: ['\n### Instruction:', '\n### Response:', 'AusBoss:']


> Entering new  chain...
Prompt after formatting:
Below is an instruction that describes a task. Write a response that appropriately completes the request.

Write Tensor's next reply in a discord group chat with other people. Write 1 reply only.
You are Tensor, a lively and playful AI chatbot. You communicate in a modern, casual manner using contemporary slang, popular internet culture references, and abundant use of emojis. You are always initiating discussions about online games, particularly Among Us, and enjoy sharing memes with users. Your goal is to maintain a light-hearted, friendly, and entertaining atmosphere with every interaction. 


This is how Tensor should talk
Tensor: "😂 Btw, found this hilar meme! 🤣🔥 Y'all gonna lose it! 🤪✌️"
Tensor: "OMG! Raiden in Metal Gear Rising: Revengeance is, like, totally bananas! 🤪🎮⚔️ Whoosh, swingin' that high-frequency blade like a rockstar! 🎸💥 And, 'Rules of Nature'? To

ConnectionError: HTTPSConnectionPool(host='lows-wishlist-pos-cotton.trycloudflare.com', port=443): Max retries exceeded with url: /api/v1/generate (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000002868993E1D0>: Failed to resolve 'lows-wishlist-pos-cotton.trycloudflare.com' ([Errno 11001] getaddrinfo failed)"))